# Combining Datasets: Merge and Join

In [111]:
import pandas as pd
import numpy as np

## Categories of Joins

The ``pd.merge()`` function implements a number of types of joins: the *one-to-one*, *many-to-one*, and *many-to-many* joins.
All three types of joins are accessed via an identical call to the ``pd.merge()`` interface; the type of join performed depends on the form of the input data.

### One-to-one joins

Perhaps the simplest type of merge expresion is the one-to-one join, which is in many ways very similar to the column-wise concatenation.

for example:

In [112]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
df1

employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

In [113]:
df2

employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

To combine this information into a single ``DataFrame``, we can use the ``pd.merge()`` function:

In [114]:
df3 = pd.merge(df1, df2)
df3

employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

### Many-to-one joins

Many-to-one joins are joins in which one of the two key columns contains duplicate entries.
For the many-to-one case, the resulting ``DataFrame`` will preserve those duplicate entries as appropriate.
Consider the following example of a many-to-one join:

In [115]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4

group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

In [116]:
df4=pd.merge(df3, df4)
df4

employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

### Many-to-many joins

Many-to-many joins are a bit confusing conceptually, but are nevertheless well defined.
If the key column in both the left and right array contains duplicates, then the result is a many-to-many merge.
This will be perhaps most clear with a concrete example.
Consider the following, where we have a ``DataFrame`` showing one or more skills associated with a particular group.
By performing a many-to-many join, we can recover the skills associated with any individual person:

In [117]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
df5

group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization

In [118]:
pd.merge(df4, df5)

employee        group  hire_date supervisor        skills
0      Bob   Accounting       2008      Carly          math
1      Bob   Accounting       2008      Carly  spreadsheets
2     Jake  Engineering       2012      Guido        coding
3     Jake  Engineering       2012      Guido         linux
4     Lisa  Engineering       2004      Guido        coding
5     Lisa  Engineering       2004      Guido         linux
6      Sue           HR       2014      Steve  spreadsheets
7      Sue           HR       2014      Steve  organization

> What if the data is not clean as practiced?

## Specification of the Merge Key

### The ``on`` keyword


### The ``left_on`` and ``right_on`` keywords


In [120]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
pd.merge(df1, df3, on="employee")

KeyError: 'employee'

In [121]:
pd.merge(df1, df3, left_on="employee", right_on="name")

employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

The result has a redundant column that we can drop if desired–for example, by using the ``drop()`` method of ``DataFrame``s:

In [122]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

employee        group  salary
0      Bob   Accounting   70000
1     Jake  Engineering   80000
2     Lisa  Engineering  120000
3      Sue           HR   90000

### The ``left_index`` and ``right_index`` keywords

Sometimes, rather than merging on a column, you would instead like to merge on an index.
For example, your data might look like this:

In [123]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')

In [124]:
df1

employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

In [125]:
df1a

group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

In [126]:
df2

employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

You can use the index as the key for merging by specifying the ``left_index`` and/or ``right_index`` flags in ``pd.merge()``:

In [127]:
df2a

hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014

In [128]:
pd.merge(df1a, df2a, left_index=True, right_index=True)

group  hire_date
employee                        
Lisa      Engineering       2004
Bob        Accounting       2008
Jake      Engineering       2012
Sue                HR       2014

For convenience, ``DataFrame``s implement the ``join()`` method, which performs a merge that defaults to joining on indices:

In [129]:
df1a.join(df2a)

group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014

__If you'd like to mix indices and columns, we can combine ``left_index`` with ``right_on`` or ``left_on`` with ``right_index`` to get the desired behavior:__

In [130]:
df3

name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

In [131]:
pd.merge(df1a, df3, left_index=True, right_on='name')

group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

## Specifying Set Arithmetic for Joins

In [132]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])

In [133]:
df6

name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

In [134]:
df7

name drink
0    Mary  wine
1  Joseph  beer

In [135]:
pd.merge(df6, df7)

name   food drink
0  Mary  bread  wine

when two datasets only have a single "name" entry in common:

By default, the result contains the *intersection* of the two sets of inputs; this is what is known as an *inner join*.

We can specify this explicitly using the ``how`` keyword, which defaults to ``"inner"``:

In [136]:
pd.merge(df6, df7, how='inner')

name   food drink
0  Mary  bread  wine

Other options for the ``how`` keyword are ``'outer'``, ``'left'``, and ``'right'``.
An *outer join* returns a join over the union of the input columns, and fills in all missing values with NAs:

In [137]:
pd.merge(df6, df7, how='outer')

name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

__The *left join* and *right join* return joins over the left entries and right entries, respectively.__
For example:

In [138]:
pd.merge(df6, df7, how='left')

name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

In [139]:
pd.merge(df6, df7, how='right')

name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer

## Overlapping Column Names: The ``suffixes`` Keyword

In [140]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
pd.merge(df8, df9, on="name")

name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

__Because the output would have two conflicting column names, the merge function automatically appends a suffix ``_x`` or ``_y`` to make the output columns unique.__

If these defaults are inappropriate, it is possible to specify a __custom suffix__ using the ``suffixes`` keyword:

In [141]:
pd.merge(df8, df9, on="name", suffixes=["_left", "_right"])

name  rank_left  rank_right
0   Bob          1           3
1  Jake          2           1
2  Lisa          3           4
3   Sue          4           2

## Example: US States Data

[Download data here](http://github.com/jakevdp/data-USstates/):

In [142]:
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

In [143]:
pop = pd.read_csv('/home/karen/Downloads/data/state-population.csv')
areas = pd.read_csv('/home/karen/Downloads/data/state-areas.csv')
abbrevs = pd.read_csv('/home/karen/Downloads/data/state-abbrevs.csv')

In [144]:
pop.head()

state/region     ages  year  population
0           AL  under18  2012     1117489
1           AL    total  2012     4817528
2           AL  under18  2010     1130966
3           AL    total  2010     4785570
4           AL  under18  2011     1125763

In [145]:
areas.head()

state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

In [146]:
abbrevs.head()

state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density.
We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.

We'll start with a many-to-one merge that will give us the full state name within the population ``DataFrame``.
We want to merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``.
We'll use ``how='outer'`` to make sure no data is thrown away due to mismatched labels.

In [147]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')

In [148]:
merged.head()

state/region     ages  year  population    state abbreviation
0           AL  under18  2012     1117489  Alabama           AL
1           AL    total  2012     4817528  Alabama           AL
2           AL  under18  2010     1130966  Alabama           AL
3           AL    total  2010     4785570  Alabama           AL
4           AL  under18  2011     1125763  Alabama           AL

In [149]:
merged.tail()

state/region     ages  year  population state abbreviation
2539          USA    total  2010   309326295   NaN          NaN
2540          USA  under18  2011    73902222   NaN          NaN
2541          USA    total  2011   311582564   NaN          NaN
2542          USA  under18  2012    73708179   NaN          NaN
2543          USA    total  2012   313873685   NaN          NaN

In [150]:
merged = merged.drop('abbreviation', 1) # drop duplicate info
merged.head()

state/region     ages  year  population    state
0           AL  under18  2012     1117489  Alabama
1           AL    total  2012     4817528  Alabama
2           AL  under18  2010     1130966  Alabama
3           AL    total  2010     4785570  Alabama
4           AL  under18  2011     1125763  Alabama

Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

In [151]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Some of the ``population`` info is null:

In [152]:
merged[merged['population'].isnull()].head()

state/region     ages  year  population state
2448           PR  under18  1990         NaN   NaN
2449           PR    total  1990         NaN   NaN
2450           PR    total  1991         NaN   NaN
2451           PR  under18  1991         NaN   NaN
2452           PR    total  1993         NaN   NaN

It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available from the original source.

More importantly, we see also that some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key!
Let's figure out which regions lack this match:

In [153]:
merged.loc[merged['state'].isnull(), 'state/region'].head()

2448    PR
2449    PR
2450    PR
2451    PR
2452    PR
Name: state/region, dtype: object

In [154]:
merged.loc[merged['state'].isnull(), 'state/region'].tail()

2539    USA
2540    USA
2541    USA
2542    USA
2543    USA
Name: state/region, dtype: object

In [155]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
We can fix these quickly by filling in appropriate entries:

In [156]:
merged.loc[merged['state/region'] == 'PR', 'state'].head()

2448    NaN
2449    NaN
2450    NaN
2451    NaN
2452    NaN
Name: state, dtype: object

In [157]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [169]:
merged.head()

state/region     ages  year  population    state
0           AL  under18  2012     1117489  Alabama
1           AL    total  2012     4817528  Alabama
2           AL  under18  2010     1130966  Alabama
3           AL    total  2010     4785570  Alabama
4           AL  under18  2011     1125763  Alabama

In [159]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

state/region     ages  year  population    state  area (sq. mi)
0           AL  under18  2012     1117489  Alabama          52423
1           AL    total  2012     4817528  Alabama          52423
2           AL  under18  2010     1130966  Alabama          52423
3           AL    total  2010     4785570  Alabama          52423
4           AL  under18  2011     1125763  Alabama          52423

Again, let's check for nulls to see if there were any mismatches:

In [160]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

There are nulls in the ``area`` column; 

> which regions were ignored?

In [161]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

We see that our ``areas`` ``DataFrame`` does not contain the area of the United States as a whole.


In [162]:
final.dropna(inplace=True)
final.head()

state/region     ages  year  population    state  area (sq. mi)
0           AL  under18  2012     1117489  Alabama          52423
1           AL    total  2012     4817528  Alabama          52423
2           AL  under18  2010     1130966  Alabama          52423
3           AL    total  2010     4785570  Alabama          52423
4           AL  under18  2011     1125763  Alabama          52423

Now we have all the data we need. To answer the question of interest, let's first select the portion of the data corresponding with the year 2000, and the total population.
We'll use the ``query()`` function to do this quickly (this requires the ``numexpr`` package to be installed; see [High-Performance Pandas: ``eval()`` and ``query()``](03.12-Performance-Eval-and-Query.ipynb)):

In [163]:
final=final.rename(columns={'area (sq. mi)':'area'})
final.head()

state/region     ages  year  population    state   area
0           AL  under18  2012     1117489  Alabama  52423
1           AL    total  2012     4817528  Alabama  52423
2           AL  under18  2010     1130966  Alabama  52423
3           AL    total  2010     4785570  Alabama  52423
4           AL  under18  2011     1125763  Alabama  52423

In [175]:
# must install numexpr first
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

state/region   ages  year  population       state    area
3             AL  total  2010     4785570     Alabama   52423
91            AK  total  2010      713868      Alaska  656425
101           AZ  total  2010     6408790     Arizona  114006
189           AR  total  2010     2922280    Arkansas   53182
197           CA  total  2010    37333601  California  163707

Now let's compute the population density and display it in order.
We'll start by re-indexing our data on the state, and then compute the result:

In [176]:
data2010.set_index('state', inplace=True)
data2010

state/region   ages  year  population    area
state                                                             
Alabama                        AL  total  2010     4785570   52423
Alaska                         AK  total  2010      713868  656425
Arizona                        AZ  total  2010     6408790  114006
Arkansas                       AR  total  2010     2922280   53182
California                     CA  total  2010    37333601  163707
Colorado                       CO  total  2010     5048196  104100
Connecticut                    CT  total  2010     3579210    5544
Delaware                       DE  total  2010      899711    1954
District of Columbia           DC  total  2010      605125      68
Florida                        FL  total  2010    18846054   65758
Georgia                        GA  total  2010     9713248   59441
Hawaii                         HI  total  2010     1363731   10932
Idaho                          ID  total  2010     1570718   83574
Illinois                       IL  total  2010    12839695   57918
Indiana                        IN  total  2010     6489965   36420
Iowa                           IA  total  2010     3050314   56276
Kansas                         KS  total  2010     2858910   82282
Kentucky                       KY  total  2010     4347698   40411
Louisiana                      LA  total  2010     4545392   51843
Maine                          ME  total  2010     1327366   35387
Maryland                       MD  total  2010     5787193   12407
Massachusetts                  MA  total  2010     6563263   10555
Michigan                       MI  total  2010     9876149   96810
Minnesota                      MN  total  2010     5310337   86943
Mississippi                    MS  total  2010     2970047   48434
Missouri                       MO  total  2010     5996063   69709
Montana                        MT  total  2010      990527  147046
Nebraska                       NE  total  2010     1829838   77358
Nevada                         NV  total  2010     2703230  110567
New Hampshire                  NH  total  2010     1316614    9351
New Jersey                     NJ  total  2010     8802707    8722
New Mexico                     NM  total  2010     2064982  121593
New York                       NY  total  2010    19398228   54475
North Carolina                 NC  total  2010     9559533   53821
North Dakota                   ND  total  2010      674344   70704
Ohio                           OH  total  2010    11545435   44828
Oklahoma                       OK  total  2010     3759263   69903
Oregon                         OR  total  2010     3837208   98386
Pennsylvania                   PA  total  2010    12710472   46058
Rhode Island                   RI  total  2010     1052669    1545
South Carolina                 SC  total  2010     4636361   32007
South Dakota                   SD  total  2010      816211   77121
Tennessee                      TN  total  2010     6356683   42146
Texas                          TX  total  2010    25245178  268601
Utah                           UT  total  2010     2774424   84904
Vermont                        VT  total  2010      625793    9615
Virginia                       VA  total  2010     8024417   42769
Washington                     WA  total  2010     6742256   71303
West Virginia                  WV  total  2010     1854146   24231
Wisconsin                      WI  total  2010     5689060   65503
Wyoming                        WY  total  2010      564222   97818
Puerto Rico                    PR  total  2010     3721208    3515

In [166]:
data2010.sort_values(by=['area', 'population'],ascending=False, inplace=True)
data2010.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


state/region   ages  year  population    area
state                                                   
Alaska               AK  total  2010      713868  656425
Texas                TX  total  2010    25245178  268601
California           CA  total  2010    37333601  163707
Montana              MT  total  2010      990527  147046
New Mexico           NM  total  2010     2064982  121593

In [167]:
data2010.sort_values(by=['population', 'area'],ascending=False, inplace=True)
data2010.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


state/region   ages  year  population    area
state                                                   
California           CA  total  2010    37333601  163707
Texas                TX  total  2010    25245178  268601
New York             NY  total  2010    19398228   54475
Florida              FL  total  2010    18846054   65758
Illinois             IL  total  2010    12839695   57918

In [172]:
data2010.head(10) # the top 10 state with most population

state/region   ages  year  population    area
state                                                       
California               CA  total  2010    37333601  163707
Texas                    TX  total  2010    25245178  268601
New York                 NY  total  2010    19398228   54475
Florida                  FL  total  2010    18846054   65758
Illinois                 IL  total  2010    12839695   57918
Pennsylvania             PA  total  2010    12710472   46058
Ohio                     OH  total  2010    11545435   44828
Michigan                 MI  total  2010     9876149   96810
Georgia                  GA  total  2010     9713248   59441
North Carolina           NC  total  2010     9559533   53821